In [1]:
import random
from tqdm import tqdm
import time
from math import sqrt

In [2]:
#L-AM FACUT SINGURA SUK MY ASS BISHEES

In [3]:
class Ant:
    def __init__(self,paramProblem):
        self.__paramProblem=paramProblem
        self.__repres=[]
        self.__pheromones=0
        self.__value=0
    
    @property
    def repres(self):
        return self.__repres
    
    @repres.setter
    def repres(self,l=[]):
        self.__repres=l
        
        
    @property
    def value(self):
        return self.__value
    
    @value.setter
    def value(self,v):
        self.__value=v
        
    @property
    def pheromones(self):
        return self.__pheromones
    
    @pheromones.setter
    def pheromones(self,pheromone):
        self.__pheromones=pheromone
    
    def __str__(self):
        return "\nAnt which has the road: \n" +  str(self.__repres) + "\n and the pheromones: \n"+ str(self.__pheromones)+ "\nwith the value \n"+ str(self.__value)+"\n"
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, c):
        return self.__repres == c.__repres

    

In [4]:

class Service:
    def __init__(self,filename):
        self.__filename=filename
        self.__paramProblem={}
        
    def get_matrix_pheromones(self):
        return self.__paramProblem['pheromone_matrix']

    def read_instruction(self):
        f = open(self.__filename, 'r')
        self.__paramProblem['noCities'] = int(f.readline())
        final_distances = []
        m_distances = []
        
        for i in range(1, self.__paramProblem['noCities'] + 1):
            distances = f.readline().split(',')
            
            for j in range(1, self.__paramProblem['noCities'] + 1):
                m_distances.append(int(distances[j - 1]))
                
            final_distances.append(m_distances)
            m_distances = []
            
        self.__paramProblem['distance_matrix'] = final_distances
        self.__paramProblem['pheromone_matrix']=[[0.3 for x in range(0,self.__paramProblem['noCities'])] for y in range(0,self.__paramProblem['noCities'])]
        
        for i in range(0, self.__paramProblem['noCities']):
            self.__paramProblem['pheromone_matrix'][i][i]=0
            
            
    def read_instruction_tsp(self):
        f=open(self.__filename,'r')
        for i in range(0,6):
            f.readline()
        cities={}
        no_cities=0
        while True:
            line= f.readline()
            if 'EOF' in line:
                break
            info=line.split(" ")
#             print(info)
            cities[int(info[0])]=(float(info[1]),float(info[2]))
            if int(info[0])>no_cities:
                no_cities=int(info[0])


        matrix=[[0 for x in range(0,no_cities)] for y in range(0,no_cities)]
        suma=0

        for i in range(0,no_cities-1):
            for j in range(i+1,no_cities):
                current_distance= sqrt((cities[i+1][0]-cities[j+1][0])**2 + (cities[i+1][1]-cities[j+1][1])**2)
                matrix[i][j]=matrix[j][i]= current_distance
                suma+=current_distance

        self.__paramProblem['noCities']= no_cities
        self.__paramProblem['distance_matrix']= matrix
        self.__paramProblem['pheromone_matrix']=[[0.3 for x in range(0,self.__paramProblem['noCities'])] for y in range(0,self.__paramProblem['noCities'])]
        
        for i in range(0, self.__paramProblem['noCities']):
            self.__paramProblem['pheromone_matrix'][i][i]=0
        
    
    
    def ant_road_and_pheromone(self):
        
        ant=Ant(self.__paramProblem)
        city=1
        ant.repres.append(1)
        
        for i in range(1,self.__paramProblem['noCities']):
            probability_ant = self.probability(ant.repres[-1],ant.repres)
            city_chosen=self.choose_probability(probability_ant)
            ant.repres.append(city_chosen)
        
        pheromone=0
        
        for i in range(0,len(ant.repres)-1):
            pheromone+=self.__paramProblem['distance_matrix'][ant.repres[i]-1][ant.repres[i+1]-1]
        
        pheromone+=self.__paramProblem['distance_matrix'][ant.repres[0]-1][ant.repres[-1]-1]
        
        ant.value=pheromone
        ant.pheromones=1/pheromone*10000
        
        return ant
    
        
    def apply_pheromones(self,ant):
            
        for i in range(0,len(ant.repres)-1):
            self.__paramProblem['pheromone_matrix'][ant.repres[i]-1][ant.repres[i+1]-1]+=ant.pheromones
            self.__paramProblem['pheromone_matrix'][ant.repres[i+1]-1][ant.repres[i]-1]+=ant.pheromones
        
        self.__paramProblem['pheromone_matrix'][ant.repres[0]-1][ant.repres[-1]-1]+=ant.pheromones
        self.__paramProblem['pheromone_matrix'][ant.repres[-1]-1][ant.repres[0]-1]+=ant.pheromones
        
        
    def choose_probability(self,probabilities):
        
       
        r = random.uniform(0.0,1.0)
        current_n=0.0
        if r==0.0:
            return probabilities[0][0]
        
        for probability in probabilities:
            if current_n<r<= probability[1]:
                return probability[0]
            current_n=probability[1]
            
    def best_3_ants(self,ants):
        
        for i in range(0, len(ants)):
            for j in range(i+1, len(ants)):
                if ants[i].value>ants[j].value:
                    aux=ants[i]
                    ants[i]=ants[j]
                    ants[j]=aux
            
        return ants[0],ants[1],ants[2]
    
    
    def probability(self,start_city,already_visited_cities):
        
        probabilities=[]
        pheromone_matrix=self.__paramProblem['pheromone_matrix']
        suma=0.0
        suma_2=0.0
        for i in range(1,self.__paramProblem['noCities']+1):
            if i!= start_city and ( i not in already_visited_cities):
                suma+=(pheromone_matrix[start_city-1][i-1]*(1/self.__paramProblem['distance_matrix'][start_city-1][i-1]))
        
        for i in range(1,self.__paramProblem['noCities']+1):
            if i!= start_city and ( i not in already_visited_cities):
                probabilities.append((i,(pheromone_matrix[start_city-1][i-1]*(1/self.__paramProblem['distance_matrix'][start_city-1][i-1]))/(suma)))
                suma_2+=(pheromone_matrix[start_city-1][i-1]*(1/self.__paramProblem['distance_matrix'][start_city-1][i-1]))/(suma)
                
        probabilities.sort(reverse=True,key=lambda x: x[1])
        new_probabilities=[]
        for i in range(0, len(probabilities)):
            suma_2-=probabilities[i][1]
            new_probabilities.append((probabilities[i][0],probabilities[i][1]+suma_2))
            
        new_probabilities.sort(reverse=False,key=lambda x: x[1])
        return new_probabilities
        
         

In [9]:
# s= Service('mediumF.txt')
# s= Service('hard_tsp_2.txt')
# s.read_instruction()
s=Service('xqf131.tsp')
s.read_instruction_tsp()

for i in tqdm(range(1000)):
    ants=[s.ant_road_and_pheromone() for x in range(0,15)]
    ant1,ant2,ant3=s.best_3_ants(ants)
    s.apply_pheromones(ant1)
    s.apply_pheromones(ant2)
    s.apply_pheromones(ant3)
    time.sleep(0.001)

print(ant1)




100%|██████████| 2500/2500 [00:47<00:00, 52.67it/s]


Ant which has the road: 
[1, 15, 12, 13, 11, 10, 14, 3, 2, 4, 5, 6, 7, 8, 9, 16, 19, 20, 18, 17, 21, 22, 26, 24, 23, 25]
 and the pheromones: 
10.21450459652707
with the value 
979

